# RQ1 — ¿Difiere el salario anual (en escala log) entre modalidades de trabajo?

## Objetivo del análisis
Para responder **RQ1** comparamos la variable objetivo transformada:

\[
\text{log\_salary}=\log\left(1+\text{ConvertedCompYearly}\right)
\]

entre las 3 categorías de `RemoteWork`:
- **Remote**
- **Hybrid**
- **In-person**

La transformación logarítmica se usa porque el salario suele tener **colas largas y outliers**, y el log ayuda a:
- estabilizar la varianza,
- reducir influencia de valores extremos,
- acercar la distribución a la normalidad (aprox.), facilitando métodos paramétricos.

---

## Por qué ANOVA es apropiado aquí
Queremos comparar la **media** de una variable numérica (`log_salary`) entre **más de dos grupos** (3 modalidades).  
El procedimiento estándar para esto es un **ANOVA de una vía (one-way ANOVA)**.

### Hipótesis
- **H0:** las medias de `log_salary` son iguales en las 3 modalidades  
  \[
  \mu_{Remote}=\mu_{Hybrid}=\mu_{In-person}
  \]
- **H1:** al menos una media difiere.

### Condiciones / supuestos del ANOVA clásico
1. **Independencia**: cada respuesta corresponde a un encuestado distinto (se asume por el diseño de la encuesta; se filtra por `ResponseId` si es necesario).
2. **Normalidad aproximada de residuos**: no es necesario que `log_salary` sea perfectamente normal; lo importante es que los **residuos** del modelo lo sean aproximadamente. La transformación log ayuda.
3. **Homoscedasticidad** (varianzas similares por grupo): puede fallar en encuestas grandes/heterogéneas.

**Decisión metodológica:**  
- Ejecutamos **ANOVA clásico** como análisis base (común y fácilmente interpretable).
- Dado que la homoscedasticidad puede no cumplirse, **confirmamos con Welch ANOVA**, que es **robusto a varianzas desiguales**.  
Si ambos coinciden, la conclusión es más sólida.

---





In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

df= pd.read_csv("../data/processed/df_final_v1.csv")
d = df.dropna(subset=["log_salary", "RemoteWork"]).copy()

# ANOVA: log_salary ~ grupo
model = smf.ols("log_salary ~ C(RemoteWork)", data=d).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table


,sum_sq,df,F,PR(>F)
C(RemoteWork),884.175795,2.0,228.977542,3.280390e-99
Residual,45224.814646,23424.0,NaN,NaN


## Preparación del dataset (población efectiva)
Como el salario tiene missingness, el análisis salarial se restringe a la subpoblación con:
- `log_salary` no nulo  
- `RemoteWork` no nulo

Esto garantiza que la comparación use únicamente participantes con información suficiente.

---

## Paso 1 — Descriptivos por grupo
Antes del contraste, se reportan **n**, **media**, **desv. estándar** y **mediana** por modalidad para:
- describir el patrón,
- contextualizar magnitud,
- anticipar posibles diferencias de dispersión.

**Resultados descriptivos (log_salary):**
- Hybrid: n=9899, media=10.8817, sd=1.2000, mediana=11.1064  
- In-person: n=3937, media=10.3539, sd=1.5120, mediana=10.7052  
- Remote: n=9591, media=10.8641, sd=1.5137, mediana=11.2253  

**Patrón inicial:**  
\[
\text{Hybrid} \approx \text{Remote} > \text{In-person}
\]

---


In [2]:
d.groupby("RemoteWork")["log_salary"].agg(["count","mean","std","median"])

,count,mean,std,median
RemoteWork,,,,
"Hybrid (some remote, some in-person)",9899,10.881707,1.200008,11.106355
In-person,3937,10.353943,1.512012,10.705198
Remote,9591,10.864146,1.513688,11.225257


## Paso 2 — ANOVA de una vía (modelo base)
Se estima el modelo:

\[
\text{log\_salary} \sim \text{RemoteWork}
\]

y se evalúa el estadístico **F**, que compara:
- variación **entre grupos** vs
- variación **dentro de grupos**

**Resultado ANOVA:**
- \(F(2, 23424)=228.98\)
- \(p=3.28\times 10^{-99}\)

**Interpretación:** se rechaza H0: **hay diferencias significativas** en la media de `log_salary` entre modalidades.

---


In [3]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(endog=d["log_salary"], groups=d["RemoteWork"], alpha=0.05)
print(tukey)


                 Multiple Comparison of Means - Tukey HSD, FWER=0.05                 
               group1                  group2  meandiff p-adj   lower   upper  reject
-------------------------------------------------------------------------------------
Hybrid (some remote, some in-person) In-person  -0.5278    0.0 -0.5891 -0.4664   True
Hybrid (some remote, some in-person)    Remote  -0.0176 0.6516 -0.0642  0.0291  False
                           In-person    Remote   0.5102    0.0  0.4486  0.5718   True
-------------------------------------------------------------------------------------


## Paso 3 — Robustez: Welch ANOVA
Como la homoscedasticidad puede ser cuestionable, se ejecuta Welch ANOVA.

**Resultado Welch:**
- \(F(2, 10297.44)=202.71\)
- \(p=4.48\times 10^{-87}\)

**Interpretación:** el resultado sigue siendo significativo bajo un método robusto, por lo que la evidencia de diferencias entre modalidades es consistente.

---


In [5]:
import pingouin as pg
pg.welch_anova(dv="log_salary", between="RemoteWork", data=d)


,Source,ddof1,ddof2,F,p-unc,np2
0,RemoteWork,2,10297.436453,202.713616,4.482265e-87,0.019176




## Paso 4 — Tamaño de efecto (importante con N grande)
Con N grande, p-valores muy pequeños pueden ocurrir incluso con diferencias pequeñas.  
Por eso se reporta tamaño de efecto:

- \(\eta_p^2 = 0.0192\)

**Interpretación:** efecto **pequeño** (≈ 1.9% de la varianza de `log_salary` explicada por modalidad).

---

## Paso 5 — Post-hoc: ¿qué pares difieren?
ANOVA/Welch dicen “alguna media difiere”, pero no especifican cuáles pares.  
Por eso se usa un post-hoc (Tukey HSD) para comparaciones múltiples.

**Tukey HSD:**
- Hybrid vs In-person: diferencia media = -0.5278, **significativo**
- Remote vs In-person: diferencia media = 0.5102, **significativo**
- Hybrid vs Remote: diferencia media = -0.0176, **no significativo** (p=0.6516)


**Conclusión post-hoc:**
- **In-person** tiene `log_salary` significativamente menor que **Remote** y **Hybrid**
- **Remote** y **Hybrid** **no** difieren significativamente entre sí.

---

## Conclusión final para RQ1
Existe evidencia estadística robusta de que el salario (en escala log) **difiere según modalidad**.  
El patrón observado es:

\[
\text{Remote} \approx \text{Hybrid} > \text{In-person}
\]

Aunque la diferencia es altamente significativa, el **tamaño de efecto es pequeño**, por lo que las diferencias, si bien consistentes, deben interpretarse con cautela y en contexto (posibles confundidores como país/experiencia/rol se abordan en RQ2).

---

## Notas y limitaciones
- El análisis describe a la subpoblación que **reportó salario** (no toda la encuesta).
- No permite inferencia causal (asociación ≠ causalidad).
- Diferencias pueden estar parcialmente explicadas por variables de confusión (Country, YearsCodePro, DevType, EdLevel), lo cual se evalúa en RQ2.
